In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.tokenize import word_tokenize
from textblob import TextBlob
nltk.download('stopwords')
import os


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gretalerer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df_running_data = pd.read_csv('../datasets/s1_laps_summary.csv')
df = pd.read_csv('../datasets/spotify_songs.csv')

In [3]:
df_running_data.head()

,event_id,timestamp,heart_rate,time_elapsed,total_running_time,altitude,distance,speed,cadence,start_time,...,total_time_spent_running_past_10800_to_14400_seconds,avg_temp,avg_temp2,avg_temp_mean,avg_lat,avg_long,avg_alt,avg_hr,total_dist,avg_temp_mode
0,1,2017-09-21T21:25:38Z,143.0,486.0,436.0,182.4,1.61317,13.2336,80.0,2017-09-21 21:17:32,...,0.0,NaN,NaN,NaN,41.966027,-87.646939,178.671811,145.828346,4.83519,NaN
1,1,2017-09-21T21:32:15Z,154.0,883.0,833.0,175.8,3.21954,14.3424,81.0,2017-09-21 21:25:39,...,0.0,NaN,NaN,NaN,41.966027,-87.646939,178.671811,145.828346,4.83519,NaN
2,1,2017-09-21T21:39:29Z,156.0,1317.0,1267.0,199.0,4.83101,12.9996,81.0,2017-09-21 21:32:16,...,0.0,NaN,NaN,NaN,41.966027,-87.646939,178.671811,145.828346,4.83519,NaN
3,2,2017-09-22T17:37:07Z,143.0,460.0,448.0,178.2,1.61227,14.5800,82.0,2017-09-22 17:29:27,...,0.0,NaN,NaN,NaN,41.975176,-87.650256,192.049602,155.442131,5.98351,NaN
4,2,2017-09-22T17:44:18Z,155.0,891.0,879.0,187.0,3.22219,13.2012,82.0,2017-09-22 17:37:08,...,0.0,NaN,NaN,NaN,41.975176,-87.650256,192.049602,155.442131,5.98351,NaN


In [4]:
df_running_data.isnull().sum()

event_id               0
timestamp             16
heart_rate            81
time_elapsed          16
total_running_time    16
                      ..
avg_long              16
avg_alt               16
avg_hr                68
total_dist            16
avg_temp_mode         80
Length: 214, dtype: int64

In [5]:
df_running_data_new = df_running_data[['avg_speed', 'avg_heart_rate', 'total_distance', 'total_running_time']]

In [6]:
df_running_data_new.head()

,avg_speed,avg_heart_rate,total_distance,total_running_time
0,13.563011,134.0,1609.34,436.0
1,13.563011,151.0,1609.34,833.0
2,13.563011,153.0,1609.34,1267.0
3,13.040247,150.0,1609.34,448.0
4,13.040247,154.0,1609.34,879.0


In [7]:
heart_rate_stats = df_running_data_new['avg_heart_rate'].describe()
low_intensity_upper_bound = heart_rate_stats['25%']  # 25th percentile
moderate_intensity_upper_bound = heart_rate_stats['75%']  # 75th percentile

print("Heart Rate Analysis:\n", heart_rate_stats)
print("\nHeart Rate Zones:")
print(f"Low Intensity: <= {low_intensity_upper_bound} BPM")
print(f"Moderate Intensity: > {low_intensity_upper_bound} and <= {moderate_intensity_upper_bound} BPM")
print(f"High Intensity: > {moderate_intensity_upper_bound} BPM")

Heart Rate Analysis:
 count    5511.000000
mean      149.862820
std        12.775517
min        60.000000
25%       141.000000
50%       150.000000
75%       160.000000
max       205.000000
Name: avg_heart_rate, dtype: float64

Heart Rate Zones:
Low Intensity: <= 141.0 BPM
Moderate Intensity: > 141.0 and <= 160.0 BPM
High Intensity: > 160.0 BPM


In [12]:
# Find the maximum heart rate in the dataset to use as a proxy for 100% MHR
max_heart_rate = df_running_data_new['avg_heart_rate'].max()

# Define heart rate zones based on percentages of MHR
def classify_refined_heart_rate_intensity(row, max_hr):
    percentage_of_max = row['avg_heart_rate'] / max_hr
    if percentage_of_max <= 0.6:
        return 'Zone 1 (Very Light)'
    elif percentage_of_max <= 0.7:
        return 'Zone 2 (Light)'
    elif percentage_of_max <= 0.8:
        return 'Zone 3 (Moderate)'
    elif percentage_of_max <= 0.9:
        return 'Zone 4 (Hard)'
    else:
        return 'Zone 5 (Maximum)'

# Apply the refined classification function to the heart rate dataset
df_running_data_new['refined_intensity_zone'] = df_running_data_new.apply(
    lambda row: classify_refined_heart_rate_intensity(row, max_heart_rate), axis=1)

# Display the first few rows of the updated heart rate dataset
df_running_data_new.head()


,avg_speed,avg_heart_rate,total_distance,total_running_time,refined_intensity_zone
0,13.563011,134.0,1609.34,436.0,Zone 2 (Light)
1,13.563011,151.0,1609.34,833.0,Zone 3 (Moderate)
2,13.563011,153.0,1609.34,1267.0,Zone 3 (Moderate)
3,13.040247,150.0,1609.34,448.0,Zone 3 (Moderate)
4,13.040247,154.0,1609.34,879.0,Zone 3 (Moderate)


In [19]:
#saving the dataset.
df_running_data_new.to_csv('../datasets/running_data.csv', index=False) 
